In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np

In [2]:
df1 = pd.read_csv("../00.data/scraped/meneame_scraped_final_1.csv", encoding="utf-8")
df2 = pd.read_csv("../00.data/scraped/meneame_scraped_final_2.csv", encoding="utf-8")
df3 = pd.read_csv("../00.data/scraped/meneame_scraped_final_3.csv", encoding="utf-8")

In [3]:
df = pd.concat([df1,df2,df3], ignore_index=True)
df

,news_id,title,content,full_story_link,meneos,clicks,karma,positive_votes,anonymous_votes,negative_votes,category,comments,published_date,user,source,source_link,provincia,comunidad,scraped_date
0,4029771,"Luigi Mangione, acusado de asesinar al CEO de ...",“Estoy abrumado y agradecido por todos los que...,https://meneame.net/story/luigi-mangione-acusa...,53,139,345,29,24,0,actualidad,17,2025-02-19 07:00:13,Verdaderofalso,infobae.com,https://www.infobae.com/estados-unidos/2025/02...,Desconocido,Desconocido,2025-02-19
1,4029544,La propaganda en la educación japonesa durante...,La historia de Japón es la historia de una nac...,https://meneame.net/story/propaganda-educacion...,40,537,362,26,14,0,cultura,1,2025-02-19 05:00:06,Charles_Dexter_Ward,newsletter.mapasmilhaud.com,https://newsletter.mapasmilhaud.com/p/propagan...,Desconocido,Desconocido,2025-02-19
2,4029786,El CEO multimillonario de Palantir no puede de...,"Alex Karp, el inquietante CEO de la inquietant...",https://meneame.net/story/ceo-multimillonario-...,104,328,398,52,52,1,actualidad,22,2025-02-19 04:05:02,Verdaderofalso,es.gizmodo.com,https://es.gizmodo.com/el-ceo-multimillonario-...,Desconocido,Desconocido,2025-02-19
3,4029797,El creador de $LIBRA alardeaba “controlar” a J...,Uno de los mensajes a los que tuvo acceso LA N...,https://meneame.net/story/creador-libra-alarde...,103,418,446,53,50,0,actualidad,26,2025-02-19 03:25:03,Suikoden,lanacion.com.ar,https://www.lanacion.com.ar/politica/escandalo...,Desconocido,Desconocido,2025-02-19
4,4029611,Uso de CRISPR para eliminar cromosomas adicion...,Las técnicas de edición genética podrían permi...,https://meneame.net/story/uso-crispr-eliminar-...,66,189,387,37,29,0,ciencia,15,2025-02-19 03:00:03,onainigo,eurekalert.org,https://www.eurekalert.org/news-releases/1073061,Desconocido,Desconocido,2025-02-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287186,6,Instalar Linux en iPod Mini | GPL Tarragona,Enseña a instalar Linux en el iPod Mini.\n\r\n...,https://meneame.net/story/instalar-linux-en-ip...,29,0,31,29,0,0,mnm,0,2005-12-07 12:00:01,asappla,gpltarragona.org,http://www.gpltarragona.org/node/view/395,Tarragona,Cataluña,2025-02-19
287187,4,Entrevista de El Mundo a Jose Antonio Marina,2. ¿Es la ética la filosofía del futuro?\r\nSi...,https://meneame.net/story/entrevista-de-el-mun...,18,0,22,18,0,0,mnm,0,2005-12-07 11:30:01,gabrielperezs,el-mundo.es,http://www.el-mundo.es/encuentros/invitados/20...,Desconocido,Desconocido,2025-02-19
287188,3,Prova xfce 4.2.3!,Ara que els de repositoris de totes les distri...,https://meneame.net/story/prova-xfce-423,34,0,13,34,0,0,mnm,0,2005-12-07 10:30:01,paurullan,xfce.org,http://www.xfce.org/release_notes/4.2.3.1_chan...,Desconocido,Desconocido,2025-02-19
287189,2,Los 84 errores de noxtrum,"Pues sí, noxtrum, el buscador de TPI y Telefón...",https://meneame.net/story/los-84-errores-de-no...,61,0,11,61,0,0,mnm,5,2005-12-07 09:25:02,FrIkI,pacoros.net,http://www.pacoros.net/diario/2005/12/07/los-8...,Desconocido,Desconocido,2025-02-19


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287191 entries, 0 to 287190
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   news_id          287191 non-null  int64 
 1   title            287191 non-null  object
 2   content          287191 non-null  object
 3   full_story_link  287191 non-null  object
 4   meneos           287191 non-null  int64 
 5   clicks           287191 non-null  int64 
 6   karma            287191 non-null  int64 
 7   positive_votes   287191 non-null  int64 
 8   anonymous_votes  287191 non-null  int64 
 9   negative_votes   287191 non-null  int64 
 10  category         287191 non-null  object
 11  comments         287191 non-null  int64 
 12  published_date   287191 non-null  object
 13  user             287191 non-null  object
 14  source           287191 non-null  object
 15  source_link      287191 non-null  object
 16  provincia        287191 non-null  object
 17  comunidad 

In [5]:
df["published_date"] = pd.to_datetime(df["published_date"])
df["scraped_date"] = pd.to_datetime(df["scraped_date"])

In [6]:
df = df.drop_duplicates(subset=["news_id"])

# Procesamiento de 'comunidad'

In [7]:
# Diccionario de provincias-ccaa
PROVINCIAS_COMUNIDADES = {
    "Andalucía": {
        "provincias": ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla"]
    },
    "Aragón": {
        "provincias": ["Huesca", "Teruel", "Zaragoza"]
    },
    "Asturias": {
        "provincias": ["Asturias"]
    },
    "Islas Baleares": {
        "provincias": ["Illes Balears"],
        "equivalencias": {
            "Mallorca": "Illes Balears",
            "Menorca": "Illes Balears",
            "Cabrera": "Illes Balears",
            "Ibiza": "Illes Balears",
            "Formentera": "Illes Balears"
        }
    },
    "Canarias": {
        "provincias": ["Las Palmas", "Santa Cruz de Tenerife"],
        "equivalencias": {
            "Gran Canaria": "Las Palmas",
            "Lanzarote": "Las Palmas",
            "Fuerteventura": "Las Palmas",
            "La Graciosa": "Las Palmas",
            "Tenerife": "Santa Cruz de Tenerife",
            "La Gomera": "Santa Cruz de Tenerife",
            "El Hierro": "Santa Cruz de Tenerife",
            "La Palma": "Santa Cruz de Tenerife"
        }
    },
    "Cantabria": {
        "provincias": ["Cantabria"]
    },
    "Castilla La Mancha": {
        "provincias": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"]
    },
    "Castilla y León": {
        "provincias": ["Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"]
    },
    "Cataluña": {
        "provincias": ["Barcelona", "Girona", "Lleida", "Tarragona"],
        "equivalencias": {
            "Gerona": "Girona",
            "Lérida": "Lleida"
        }
    },
    "Extremadura": {
        "provincias": ["Badajoz", "Cáceres"]
    },
    "Galicia": {
        "provincias": ["A Coruña", "Lugo", "Ourense", "Pontevedra"],
        "equivalencias": {
            "Coruña": "A Coruña"
        }
    },
    "Madrid": {
        "provincias": ["Madrid"]
    },
    "Murcia": {
        "provincias": ["Murcia"]
    },
    "Navarra": {
        "provincias": ["Navarra"]
    },
    "La Rioja": {
        "provincias": ["La Rioja"]
    },
    "País Vasco": {
        "provincias": ["Araba/Álava", "Bizkaia/Vizcaya", "Gipuzkoa/Guipúzcoa"],
        "equivalencias": {
            "Araba": "Araba/Álava",
            "Álava": "Araba/Álava",
            "Bizkaia": "Bizkaia/Vizcaya",
            "Vizcaya": "Bizkaia/Vizcaya",
            "Gipuzkoa": "Gipuzkoa/Guipúzcoa",
            "Guipúzcoa": "Gipuzkoa/Guipúzcoa"
        }
    },
    "Comunidad Valenciana": {
        "provincias": ["Alacant/Alicante", "Castelló/Castellón", "València/Valencia"],
        "equivalencias": {
            "Alicante": "Alacant/Alicante",
            "Alacant": "Alacant/Alicante",
            "Castellón": "Castelló/Castellón",
            "Castelló": "Castelló/Castellón",
            "Valencia": "València/Valencia",
            "València": "València/Valencia"
        }
    },
    "Ceuta": {
        "provincias": ["Ceuta"]
    },
    "Melilla": {
        "provincias": ["Melilla"]
    }
}

In [8]:
# función para asignar provincias y ccaa del title/content
def asignar_provincia_comunidad(row):
    for comunidad, datos in PROVINCIAS_COMUNIDADES.items():
        provincias = datos["provincias"]
        equivalencias = datos.get("equivalencias", {})
        
        for provincia in provincias:
            if provincia in row["title"] or provincia in row["content"]:
                return {"provincia": provincia, "comunidad": comunidad}
        
        for variante, provincia_estandar in equivalencias.items():
            if variante in row["title"] or variante in row["content"]:
                return {"provincia": provincia_estandar, "comunidad": comunidad}

    return {"provincia": "Desconocido", "comunidad": "Desconocido"}

In [9]:
# Aplicar la función fila por fila
df[["provincia", "comunidad"]] = df.apply(asignar_provincia_comunidad, axis=1).apply(pd.Series)

In [10]:
df["comunidad"].value_counts()

comunidad
Desconocido             239683
Madrid                   14380
Andalucía                 6524
Comunidad Valenciana      5531
Cataluña                  5432
Castilla y León           3885
Castilla La Mancha        1586
Galicia                   1567
Aragón                    1366
Canarias                  1320
Murcia                    1232
Asturias                  1138
Islas Baleares            1036
Navarra                    540
País Vasco                 532
Extremadura                488
Cantabria                  407
Ceuta                      240
Melilla                    158
La Rioja                   146
Name: count, dtype: int64

In [11]:
df['comunidad'] = df['comunidad'].replace('Desconocido', np.nan)

df['provincia'] = df['provincia'].replace('Desconocido', np.nan)



# Procesamiento de Categorias


In [12]:
news_category_map = {
    "EEUU": "Política y Sociedad",
    'MADRID': "Política y Sociedad",
    "Rusia": "Política y Sociedad",
    "Japón": "Política y Sociedad",
    "México": "Política y Sociedad",
    "Venezuela": "Política y Sociedad",
    "Cataluña": "Política y Sociedad",
    "Galiza": "Política y Sociedad",
    "Andalucía": "Política y Sociedad",
    "Balears": "Política y Sociedad",
    "Canarias": "Política y Sociedad",
    "CastillaLeón": "Política y Sociedad",
    "deutschland": "Política y Sociedad",
    "Siria": "Conflictos y Guerra",
    "Palestina": "Política y Sociedad",
    "Israel": "Política y Sociedad",
    "Africa": "Política y Sociedad",
    "OrienteMedio": "Política y Sociedad",
    "Cuba": "Política y Sociedad",
    "Barcelona": "Política y Sociedad",
    "Galicia": "Política y Sociedad",
    "Cantabria": "Política y Sociedad",
    "Aragón": "Política y Sociedad",
    "Málaga": "Política y Sociedad",
    "Roma": "Política y Sociedad",
    "INTERNAC": "Política y Sociedad",
    "politica": "Politica y Sociedad",
    "corruPPción": "Politica y Sociedad",
    "EleccionesE": "Politica y Sociedad",
    "Podemos": "Politica y Sociedad",
    "República": "Politica y Sociedad",
    "RentaBásica": "Politica y Sociedad",
    "Cayetanos": "Politica y Sociedad",
    "corruptcion": "Politica y Sociedad",
    "LuchaDClases": "Politica y Sociedad",
    "Anarquismo": "Politica y Sociedad",
    "DDHH": "Politica y Sociedad",
    "Autonomos": "Politica y Sociedad",
    "Justicia7291": "Politica y Sociedad",
    "Tribunales": "Politica y Sociedad",
    "actualidad": "Politica y Sociedad",
    "ZasSmeame": "Politica y Sociedad",
    "Campañas": "Politica y Sociedad",
    "corrupcion": "Politica y Sociedad",
    "Fachéame": "Politica y Sociedad",
    "VIOLENCIA_G": "Politica y Sociedad",
    "prensa": "Politica y Sociedad",
    "borbones": "Politica y Sociedad",
    "KKKantiPDMS": "Politica y Sociedad",
    "autonomos": "Politica y Sociedad",
    "AEDE": "Politica y Sociedad",
    "procescatalà": "Politica y Sociedad",
    "RebeliónMnm": "Politica y Sociedad",
    "DchoLaboral": "Política y Sociedad",
    "Bulos": "Política y Sociedad",
    "Desinfórmame": "Política y Sociedad",
    "Madriléame": "Política y Sociedad",
    "Begoñéame": "Política y Sociedad",
    
    "tecnología": "Tecnologia y Ciencia",
    "Informática": "Tecnologia y Ciencia",
    "GNU_LINUX": "Tecnologia y Ciencia",
    "linux": "Tecnologia y Ciencia",
    "startup": "Tecnologia y Ciencia",
    "Criptomoneda": "Tecnologia y Ciencia",
    "bitcoin": "Tecnologia y Ciencia",
    "SectorTIC": "Tecnologia y Ciencia",
    "Meteorología": "Tecnologia y Ciencia",
    "Física": "Tecnologia y Ciencia",
    "genetica": "Tecnologia y Ciencia",
    "Neurociencia": "Tecnologia y Ciencia",
    "Debian": "Tecnologia y Ciencia",
    "retrocomp": "Tecnologia y Ciencia",
    "hacking": "Tecnologia y Ciencia",
    "Metamaterial": "Tecnologia y Ciencia",
    "Micromundo": "Tecnologia y Ciencia",
    "atomo": "Tecnologia y Ciencia",
    "Impresión3D": "Tecnologia y Ciencia",
    "Drones": "Tecnologia y Ciencia",
    "netsec": "Tecnologia y Ciencia",
    "RaspberryPi": "Tecnologia y Ciencia",
    "SysDevs": "Tecnologia y Ciencia",
    "ciencia": "Tecnologia y Ciencia",
    "astronomia": "Tecnologia y Ciencia",
    "ASTRONÁUTICA": "Tecnologia y Ciencia",
    "softlibre": "Tecnologia y Ciencia",
    "Ubuntu": "Tecnologia y Ciencia",
    "android": "Tecnologia y Ciencia",
    "smartphone": "Tecnologia y Ciencia",
    "Firefox": "Tecnologia y Ciencia",
    "Blender": "Tecnologia y Ciencia",
    "IOOcia": "Tecnologia y Ciencia",
     "Marte": "Tecnologia y Ciencia",
    "Visualdata": "Tecnologia y Ciencia",
    "INVENTOS": "Tecnologia y Ciencia",
    "Numismática": "Tecnologia y Ciencia",
    "FÓSILES": "Tecnologia y Ciencia",
    
    
    
    "salud": "Salud y Medicina",
    "medicina": "Salud y Medicina",
    "Nutrición": "Salud y Medicina",
    "sanidad": "Salud y Medicina",
    "PREVENCIÓN": "Salud y Medicina",
    "HOMEOPATÍA": "Salud y Medicina",
    "VacunaGate": "Salud y Medicina",
    "vacúname": "Salud y Medicina",
    "SANIDAD": "Salud y Medicina",
    "Alimentación": "Salud y Medicina", 
    "ebola": "Salud y Medicina",
    "Coronavirus": "Salud y Medicina",

    "Cachopos": "Gastronomía",
    "cervecéame": "Gastronomía",
    
    "Economía": "Negocios y Economia",
    "banca": "Negocios y Economia",
    "subvenciones": "Negocios y Economia",
    "CrisEnergéti": "Negocios y Economia",
    
    "ocio": "Entretenimiento y Cultura",
    "Cine": "Entretenimiento y Cultura",
    "Series": "Entretenimiento y Cultura",
    "Videojuegos": "Entretenimiento y Cultura",
    "Fotografía": "Entretenimiento y Cultura",
    "Música": "Entretenimiento y Cultura",
    "cinéfilos": "Entretenimiento y Cultura",
    "FolkloreÑ": "Entretenimiento y Cultura",
    "Cómics": "Entretenimiento y Cultura",
    "Webcomics": "Entretenimiento y Cultura",
    "ArteyMercado": "Entretenimiento y Cultura",
    "Arte": "Entretenimiento y Cultura",
    "Ilustracion": "Entretenimiento y Cultura",
    "clásica": "Entretenimiento y Cultura",
    "MUSICA70": "Entretenimiento y Cultura",
    "PALABROS": "Entretenimiento y Cultura",
    "Covers": "Entretenimiento y Cultura",
    "cultura": "Entretenimiento y Cultura",
    "Eurovision": "Entretenimiento y Cultura",
    "Poesia": "Entretenimiento y Cultura",
    "Bushido": "Entretenimiento y Cultura",
    "Arquitectura": "Entretenimiento y Cultura",
    "rol": "Entretenimiento y Cultura",
    "Podcasteros": "Entretenimiento y Cultura",
    "Viajar": "Entretenimiento y Cultura",
    "CiFi": "Entretenimiento y Cultura",
    "Gastronomia": "Entretenimiento y Cultura",
    "Clicómics": "Entretenimiento y Cultura",
    "MUSICA70":"Entretenimiento y Cultura",
    "Cocíname": "Entretenimiento y Cultura",
    "curiosidades": "Entretenimiento y Cultura",
    "acapela": "Entretenimiento y Cultura", 
    "ilustracion": "Entretenimiento y Cultura", 
    "twitter":"Entretenimiento y Cultura", 
    "reddit":"Entretenimiento y Cultura", 
    "museo": "Entretenimiento y Cultura", 
    "Relatocorto": "Entretenimiento y Cultura",
    "Autorrelatos": "Entretenimiento y Cultura",
    "Reportajes": "Entretenimiento y Cultura",
    "Entrevistas": "Entretenimiento y Cultura",
    "microrelatos": "Entretenimiento y Cultura",
    "Hazlotumismo": "Entretenimiento y Cultura",
    "Los12monos": "Entretenimiento y Cultura",
    "Pregúntame" : "Entretenimiento y Cultura",
    "Recomiéndame": "Entretenimiento y Cultura",
    "LETREROS": "Entretenimiento y Cultura",
    "Escúchame": "Entretenimiento y Cultura",
    "Documentales": "Entretenimiento y Cultura",
    "MÚSICA70": "Entretenimiento y Cultura",
    "STARWARS": "Entretenimiento y Cultura",
    "Ochenteando": "Entretenimiento y Cultura",
    "UHF": "Entretenimiento y Cultura",
    "Preguntas": "Entretenimiento y Cultura",
    "Fotomundo": "Entretenimiento y Cultura",
    "roleros": "Entretenimiento y Cultura",
    "metal": "Entretenimiento y Cultura",
    "B.S.O.": "Entretenimiento y Cultura",
    "COVERS": "Entretenimiento y Cultura",
    "TEMAZOS": "Entretenimiento y Cultura",
    "Comiñas": "Entretenimiento y Cultura",
    "MundoCelta": "Entretenimiento y Cultura",
    "Artículos": "Entretenimiento y Cultura",
    "sabíasque": "Entretenimiento y Cultura",
    "paléame": "Entretenimiento y Cultura",

    "deportes": "Deportes",
    "Natación": "Deportes",
    "Motociclismo": "Deportes",
    "ciclismo": "Deportes",

    "veganismo": "Medioambiente y Energia",
    "MenteAnimal": "Medioambiente y energía",
    "Abandonos": "Medioambiente y energía",
    "MUNDO_PROTE": "Medioambiente y energía",
    "MAmbiente": "Medioambiente y Energia",
    "Cambioclima": "Medioambiente y Energia",
    "Ecología": "Medioambiente y Energia",
    "Energias": "Medioambiente y Energia",
    "sequía": "Medioambiente y Energia",
    "Naturaleza": "Medioambiente y Energia",
    "Botánica": "Medioambiente y Energia",
    "PLANTAS": "Medioambiente y Energia",
    "AgroInfo": "Medioambiente y Energia",
    "Vivienda": "Medioambiente y Energia",
    "Animales": "Medioambiente y Energia",
    "gatos": "Medioambiente y Energia",
    "Biologia": "Medioambiente y Energia",
    "Faros": "Medioambiente y Energia",
    "mascotas": "Medioambiente y Energia",
    "Medioambiente y energía": "Medioambiente y Energia",
    "Cáñamo": "Medioambiente y Energia",
    "Arañas": "Medioambiente y Energia",
    
    "Historia": "Historia y Humanidades",
    "Mitología": "Historia y Humanidades",
    "Arqueología": "Historia y Humanidades",
    "Egiptologia": "Historia y Humanidades",
    "Viñetas": "Historia y Humanidades",
    "Hallazgos": "Historia y Humanidades",
    "Asturias": "Historia y Humanidades",
    "Jurídicas": "Historia y Humanidades",
    "Etimologia": "Historia y Humanidades",
    "Psicología": "Historia y Humanidades",
    "Psicópatas": "Historia y Humanidades",
    "Filosofía": "Historia y Humanidades",
    "egiptologia": "Historia y Humanidades",
    "Efemérides": "Historia y Humanidades",
    "Helénica": "Historia y Humanidades", 
    "Hemeroteca": "Historia y Humanidades",
    "Fragmentos": "Historia y Humanidades",

    
    
    "Humor": "Humor y Memes",
    "memes": "Humor y Memes",
    "Sarcasmos": "Humor y Memes",
    "Chorridolias": "Humor y Memes",
    "Tendenciosos": "Humor y Memes",
    "Ayuséame": "Humor y Memes",
    "podríame": "Humor y Memes",
    "Fakeame": "Humor y Memes",
    "Fakejóo": "Humor y Memes",
    "Idiocracia": "Humor y Memes",
    "meneametoday": "Humor y Memes",
    "TuMeme": "Humor y Memes",
    "Cosasguays": "Humor y Memes",
    "friki": "Humor y Memes",
    "procrastrina": "Humor y Memes",
    "NoMundoToday": "Humor y Memes",
    "Memeneame": "Humor y Memes",
    "Ardilléame": "Humor y Memes",
    "Toréame": "Humor y Memes",
    "OYOYOY": "Humor y Memes",
    "gilipolleces": "Humor y Memes",
    "chorradas": "Humor y Memes",

    "GuerraCivilñ": "Conflictos y Guerra",
    "ayudaucrania": "Conflictos y Guerra",
    "Rescates": "Conflictos y Guerra",
    
    "aviacion": "Transporte",
    "aviones": "Transporte",
    "trenes": "Transporte",
    "DIRIGIBLES": "Transporte",
    "Submarino": "Transporte",
    "Motor": "Transporte",
    "Avionéame": "Transporte",
    "BARCOS": "Transporte",
    "Tractores": "Transporte",
    "movilidad": "Transporte",
    "alas": "Transporte", 
    
    "Educación": "Educacion",
    "idiomas": "Educacion",
    "Enseñanza": "Educacion",
    "Literatura": "Educacion",
    "Libros": "Educacion",
    "lenguas": "Educacion",
    "Matemáticas": "Educacion",
    
    "INV": "Crimen",
    "Gamonal": "Crimen",
    "Sectas": "Crimen",
    "Sucesos": "Crimen",

    "Asperger": "Cuestiones Sociales",
    "Crianza": "Cuestiones Sociales",
    "Drogolegas": "Cuestiones Sociales",
    "Feminismo": "Cuestiones Sociales",
    "LGBT": "Cuestiones Sociales",
    "Derecho": "Cuestiones Sociales",
    "SEXOLOGÍA": "Cuestiones Sociales",
    "mujeresenlah": "Cuestiones Sociales",
    "Abuso_Animal": "Cuestiones Sociales",
    "religion": "Cuestiones Sociales",
    "laicismo": "Cuestiones Sociales",
}

In [13]:
def category_mapping(category):
    return news_category_map.get(category, "Miscelaneo")

df["category"] = df["category"].apply(category_mapping)

In [14]:
df["category"].value_counts()

category
Miscelaneo                   135837
Politica y Sociedad           94930
Entretenimiento y Cultura     40406
Tecnologia y Ciencia          14485
Historia y Humanidades          294
Humor y Memes                   239
Política y Sociedad             235
Transporte                      220
Medioambiente y Energia         198
Salud y Medicina                137
Crimen                           74
Cuestiones Sociales              69
Negocios y Economia              22
Educacion                        19
Deportes                         13
Conflictos y Guerra               8
Medioambiente y energía           3
Gastronomía                       2
Name: count, dtype: int64

In [15]:
df

,news_id,title,content,full_story_link,meneos,clicks,karma,positive_votes,anonymous_votes,negative_votes,category,comments,published_date,user,source,source_link,provincia,comunidad,scraped_date
0,4029771,"Luigi Mangione, acusado de asesinar al CEO de ...",“Estoy abrumado y agradecido por todos los que...,https://meneame.net/story/luigi-mangione-acusa...,53,139,345,29,24,0,Politica y Sociedad,17,2025-02-19 07:00:13,Verdaderofalso,infobae.com,https://www.infobae.com/estados-unidos/2025/02...,NaN,NaN,2025-02-19
1,4029544,La propaganda en la educación japonesa durante...,La historia de Japón es la historia de una nac...,https://meneame.net/story/propaganda-educacion...,40,537,362,26,14,0,Entretenimiento y Cultura,1,2025-02-19 05:00:06,Charles_Dexter_Ward,newsletter.mapasmilhaud.com,https://newsletter.mapasmilhaud.com/p/propagan...,NaN,NaN,2025-02-19
2,4029786,El CEO multimillonario de Palantir no puede de...,"Alex Karp, el inquietante CEO de la inquietant...",https://meneame.net/story/ceo-multimillonario-...,104,328,398,52,52,1,Politica y Sociedad,22,2025-02-19 04:05:02,Verdaderofalso,es.gizmodo.com,https://es.gizmodo.com/el-ceo-multimillonario-...,NaN,NaN,2025-02-19
3,4029797,El creador de $LIBRA alardeaba “controlar” a J...,Uno de los mensajes a los que tuvo acceso LA N...,https://meneame.net/story/creador-libra-alarde...,103,418,446,53,50,0,Politica y Sociedad,26,2025-02-19 03:25:03,Suikoden,lanacion.com.ar,https://www.lanacion.com.ar/politica/escandalo...,NaN,NaN,2025-02-19
4,4029611,Uso de CRISPR para eliminar cromosomas adicion...,Las técnicas de edición genética podrían permi...,https://meneame.net/story/uso-crispr-eliminar-...,66,189,387,37,29,0,Tecnologia y Ciencia,15,2025-02-19 03:00:03,onainigo,eurekalert.org,https://www.eurekalert.org/news-releases/1073061,NaN,NaN,2025-02-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287186,6,Instalar Linux en iPod Mini | GPL Tarragona,Enseña a instalar Linux en el iPod Mini.\n\r\n...,https://meneame.net/story/instalar-linux-en-ip...,29,0,31,29,0,0,Miscelaneo,0,2005-12-07 12:00:01,asappla,gpltarragona.org,http://www.gpltarragona.org/node/view/395,Tarragona,Cataluña,2025-02-19
287187,4,Entrevista de El Mundo a Jose Antonio Marina,2. ¿Es la ética la filosofía del futuro?\r\nSi...,https://meneame.net/story/entrevista-de-el-mun...,18,0,22,18,0,0,Miscelaneo,0,2005-12-07 11:30:01,gabrielperezs,el-mundo.es,http://www.el-mundo.es/encuentros/invitados/20...,NaN,NaN,2025-02-19
287188,3,Prova xfce 4.2.3!,Ara que els de repositoris de totes les distri...,https://meneame.net/story/prova-xfce-423,34,0,13,34,0,0,Miscelaneo,0,2005-12-07 10:30:01,paurullan,xfce.org,http://www.xfce.org/release_notes/4.2.3.1_chan...,NaN,NaN,2025-02-19
287189,2,Los 84 errores de noxtrum,"Pues sí, noxtrum, el buscador de TPI y Telefón...",https://meneame.net/story/los-84-errores-de-no...,61,0,11,61,0,0,Miscelaneo,5,2005-12-07 09:25:02,FrIkI,pacoros.net,http://www.pacoros.net/diario/2005/12/07/los-8...,NaN,NaN,2025-02-19


In [16]:
# Guardar en partes para poder subir a Git archivos menores 100MB
df1 = df.iloc[:100_000]
df2 = df.iloc[100_000:200_000]
df3 = df.iloc[200_000:]

df1.to_csv("../00.data/preprocesado/meneame_procesado_1.csv", index=False, encoding="utf-8")
df2.to_csv("../00.data/preprocesado/meneame_procesado_2.csv", index=False, encoding="utf-8")
df3.to_csv("../00.data/preprocesado/meneame_procesado_3.csv", index=False, encoding="utf-8")